In [1]:
import pandas as pd

In [2]:
import os
import sys

# add path
if '../..' in sys.path:
    pass
else:
    sys.path.insert(0, '../..')
    

from loader.greenhouse_loader import GreenhouseLoader
from loader.power_industry_loader import PowerLoader

def load_co2_data():
    ghg_loader = GreenhouseLoader()
    cur_dir = os.getcwd()
    os.chdir('../../')
    ghg_loader.load()
    df_ghg = ghg_loader.get_data()
    os.chdir(cur_dir)
    return df_ghg

def load_power_industry():
    power_loader = PowerLoader()
    cur_dir = os.getcwd()
    os.chdir('../../')
    power_loader.load()
    df_power = power_loader.get_data()
    os.chdir(cur_dir)
    return df_power
    

In [3]:
gg_data = load_co2_data()
co2_df = gg_data['co2_country_sector']['Power Industry']
co2_df.index = pd.to_datetime(co2_df.index)
co2_df = co2_df.rename(columns={'Spain and Andorra': 'Spain',
                                'France and Monaco': 'France',
                                'Italy, San Marino and the Holy See': 'Italy',
                                'Switzerland and Liechtenstein': 'Switzerland'})
co2_df = co2_df.loc['2008':, :]
co2_df.columns

Index(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Anguilla',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia',
       ...
       'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela', 'Vietnam',
       'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe', 'EU'],
      dtype='object', name='country_name', length=208)

In [14]:
power_data = load_power_industry()

EU_co2 = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
                    'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
                    'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
                    'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom']

supply_df = power_data['supply']
supply_df.name = 'Supply'
supply_forecast = supply_df['2019':'2020-04-01']
supply_df = supply_df[:'2018']
supply_df['EU'] = supply_df.loc[:, EU_co2].sum(axis=1)
supply_forecast = supply_df['2019':'2020-04-01']
supply_df = supply_df[:'2018']

oil_df = power_data['oil']
oil_df.name = 'Oil'
oil_forecast = oil_df['2019':'2020-04-01']
oil_df = oil_df[:'2018']
oil_df['EU'] = oil_df.loc[:, EU_co2].sum(axis=1)
oil_forecast = oil_df['2019':'2020-04-01']
oil_df = oil_df[:'2018']

brent_df = power_data['brent']
brent_df = brent_df.loc['2008':]
brent_df = brent_df.rename(columns={'Price': 'Brent price'})
brent_forecast = brent_df['2019':'2020-04-01']
brent_df = brent_df[:'2018']

gas_df = power_data['natural_gas']
gas_df = gas_df.loc['2008':]
gas_df = gas_df.rename(columns={'Price': 'Natural gas price'})
gas_forecast = gas_df['2019':'2020-04-01']
gas_df = gas_df[:'2018']

supply_forecast

<ipython-input-14-b67ffae69f62>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supply_df['EU'] = supply_df.loc[:, EU_co2].sum(axis=1)
<ipython-input-14-b67ffae69f62>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil_df['EU'] = oil_df.loc[:, EU_co2].sum(axis=1)


,Belgium,Bulgaria,Czechia,Denmark,Germany,Estonia,Ireland,Greece,Spain,France,...,Romania,Slovenia,Slovakia,Finland,Sweden,United Kingdom,Norway,North Macedonia,Turkey,EU


In [13]:
models = {}
for country in supply_df.columns:
    data = pd.concat([brent_df, gas_df, supply_df[country].rename('Supply'), oil_df[country].rename('Oil'), co2_df[country].rename('CO2')], axis=1)
    forecast = pd.concat([brent_forecast, gas_forecast, supply_forecast[country].rename('Supply'), oil_forecast[country].rename('Oil')], axis=1)
    models[country] = {'data': data, 'forecast': forecast}

models['Spain']

{'data':             Brent price  Natural gas price  Supply     Oil     CO2
 2008-01-01        92.18               7.99   114.6  6689.0  102.68
 2008-02-01        94.99               8.54   116.0  6266.0     NaN
 2008-03-01       103.64               9.41   118.3  6515.0     NaN
 2008-04-01       109.07              10.18   120.5  6540.0     NaN
 2008-05-01       122.80              11.27   117.6  6540.0     NaN
 ...                 ...                ...     ...     ...     ...
 2018-08-01        72.53               2.96    99.3  5456.0     NaN
 2018-09-01        78.89               3.00    95.9  5265.0     NaN
 2018-10-01        81.03               3.28    98.1  5570.0     NaN
 2018-11-01        64.75               4.09    96.7  5389.0     NaN
 2018-12-01        57.36               4.04    92.5  5339.0     NaN
 
 [132 rows x 5 columns],
 'forecast':             Brent price  Natural gas price  Supply  Oil
 2019-01-01        59.41               3.11     NaN  NaN
 2019-02-01        63.9

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

def scale(df):
    df_scaled = pd.DataFrame(scaler.fit_transform(df))
    df_scaled.index = df.index
    df_scaled.columns = df.columns
    return df_scaled

plt.figure()
ax = plt.gca()
 = scale(df_steel)
concrete_norm = scale(df_concrete)
employ_norm = scale(df_employ)
co2_norm = scale(pd.DataFrame(co2_sum))
co2_norm.columns = ['CO2 Emissions Construction Sector']
steel_norm.plot(ax=ax)
concrete_norm.plot(ax=ax)
employ_norm.plot(ax=ax)
co2_norm.plot(ax=ax)
#co2_sum.plot(ax=ax)